In [121]:
import duckdb
import polars as pl

today = "2023-12-31"
procurement_window = 6

with duckdb.connect("./data/duckdb.db") as con:
    df_demand_predictions = con.execute(
        """
        SELECT
            dp_p_id as p_id,
            dp_s_id as s_id,
            dp_period as period,
            dp_mean as mean
        FROM demand_predictions""").pl()
    df_procurements = con.execute(
        """
        SELECT 
            pc_p_id as p_id,
            pc_s_id as s_id,
            pc_active_from,
            pc_active_upto
        FROM procurements""").pl()
    df_transport_links = con.execute(
        """
        SELECT 
            tl_w_id as w_id,
            tl_s_id as s_id,
            tl_cost as cost
        FROM transport_links""").pl()
    df_current_stocks = con.execute(
        f"""
        SELECT
            sk_p_id as p_id, 
            sk_s_id as s_id, 
            0 as period_rank, 
            sk_ending_inventory as ending_inventory,
        FROM stocks
        WHERE sk_period == DATE '{today}'""").pl()
    df_periods = con.execute(f"""
        SELECT 
            CAST(d AS DATE) AS period,
            ROW_NUMBER() OVER (ORDER BY d) - 1 AS period_rank
        FROM generate_series(
            DATE '{today}', 
            DATE '{today}' + INTERVAL {procurement_window} DAY, 
            INTERVAL 1 DAY
        ) AS t(d)
    """).pl()

In [130]:
df_demand_predictions

p_id,s_id,mean,period_rank
i32,i32,i32,i64
1,2,10,1
1,2,10,2
1,2,10,3
1,2,5,4
1,2,5,5
…,…,…,…
5,3,10,2
5,3,7,3
5,3,10,4


In [129]:
df_procurements

p_id,s_id,period_rank
i32,i32,i64
1,2,1
1,2,2
1,2,3
1,2,4
1,2,5
…,…,…
4,1,6
4,2,6
4,3,6


In [124]:
df_transport_links

w_id,s_id,cost
i32,i32,f64
1,1,1.0
1,2,3.0
1,3,9.0
2,1,3.0
2,2,9.0
2,3,1.0


In [125]:
df_current_stocks

p_id,s_id,period_rank,ending_inventory
i32,i32,i32,i32
3,1,0,5
3,3,0,5
2,1,0,10
2,2,0,10
2,3,0,5
…,…,…,…
4,1,0,5
4,2,0,5
4,3,0,0


In [88]:
# from datetime import date, timedelta

# today =  date.fromisoformat("2023-12-31")
# procurement_window = 6
# start_procurement = today + timedelta(days=1)
# end_procurement = today + timedelta(days=procurement_window)

# # create a df_periods with the dates we want to procurement for from tomorrow upto tomorrow + procurement window
# df_periods = pl.DataFrame()

# df_periods = df_periods.with_columns(
#     period = pl.date_range(
#         start=today, 
#         end=end_procurement,
#         interval="1d", closed="both"
#     ).cast(pl.Date),
#     period_rank=pl.arange(0, procurement_window+1).cast(pl.Int32),
# )

In [126]:
# do a join to get the periods where the assignments are active
import polars.selectors as cs

df_procurements = df_procurements.join_where(
    df_periods,
    (pl.col('pc_active_from') <= pl.col("period")) & \
    (pl.col("period") < pl.col('pc_active_upto')) & \
    (pl.col("period_rank") > 0)
).drop(cs.date())
    #['pc_active_from', 'pc_active_upto', "period"])

In [127]:
# df_procurements.sort(by=["p_id", "s_id", "period_rank"]).to_dicts()

In [128]:
df_demand_predictions = df_demand_predictions.join(df_periods, on="period", how="left") \
.drop(cs.date())

In [ ]:
# df_ending_inventory = 
# df_stocks.join_where(df_periods, pl.col("period_rank") > 0)#.drop(["sk_p_id", "sk_s_id", "period_rank"])

p_id,s_id,period,ending_inventory,period_right,period_rank
i32,i32,date,i32,date,i64
3,1,2023-12-31,5,2024-01-01,1
3,1,2023-12-31,5,2024-01-02,2
3,1,2023-12-31,5,2024-01-03,3
3,1,2023-12-31,5,2024-01-04,4
3,1,2023-12-31,5,2024-01-05,5
…,…,…,…,…,…
4,1,2023-12-31,5,2024-01-06,6
4,2,2023-12-31,5,2024-01-06,6
4,3,2023-12-31,0,2024-01-06,6


In [119]:
# df_starting_inventory = df_stocks.join(
#     df_periods, 
#     on="period", 
#     how="left"
# ).drop(cs.date())
# df_starting_inventory

In [131]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()
solver = cp_model.CpSolver()

def polars_to_dict(df, key_cols, value_col):
    """
    Convierte un DataFrame de Polars a un diccionario con tuplas como clave.
    key_cols: lista de nombres de columnas para la clave.
    value_col: nombre de la columna para el valor.
    """
    return {
        tuple(row[col] for col in key_cols): row[value_col]
        for row in df.to_dicts()
    }


def empty_dict_from_polars(df, key_cols=None):
    if not key_cols:
        key_cols = df.columns
    return {tuple(row): None for row in zip(*[df[col] for col in key_cols])}

In [132]:
new_orders = empty_dict_from_polars(df_procurements)
for p, s, t in new_orders.keys():
    new_orders[p,s,t] = model.new_int_var(0, 1000, f"new_order_{p}_{s}_{t}")

In [134]:

demand_predictions = polars_to_dict(df_demand_predictions, key_cols=["p_id", "s_id", "period_rank"], value_col="mean")
met_demand = demand_predictions.copy() #empty_dict_from_polars(df_demand_predictions.drop("mean")) #key_cols=["p_id", "s_id", "period_rank"])
unmet_demand = demand_predictions.copy()

for p, s, t in demand_predictions.keys():
    met_demand[p,s,t] = model.new_int_var(0, 1000, f"met_demand_{p}_{s}_{t}")
    unmet_demand[p,s,t] = model.new_int_var(0, 1000, f"unmet_demand_{p}_{s}_{t}")

In [135]:
current_stocks = polars_to_dict(df_current_stocks, key_cols=["p_id", "s_id"], value_col="ending_inventory")
expected_stocks = empty_dict_from_polars(df_procurements)
for p, s, t in expected_stocks.keys():
    expected_stocks[p,s,t] = model.new_int_var(0, 1000, f"ending_inv_{p}_{s}_{t}")

In [136]:
# define constraints for met and unmet demand
for p, s, t in demand_predictions.keys():
    # met demand is the sum of all orders for product p, location l, and time t
    # model.add(met_demand[p,l,t] == sum(order_vars[p, l, t, o] for o in vars_tuples.o))
    
    # met demand should not exceed forecast demand
    # model.add(met_demand[p,s,t] <= demand_predictions[p,s,t])
    
    # unmet demand is the difference between demand and met demand
    model.add(unmet_demand[p,s,t] + met_demand[p,s,t] == demand_predictions[p,s,t])
    
    # # new orders should be at least the unmet demand
    # model.add(new_orders[p,l,t] >= unmet_demand[p,l,t])

In [137]:
#create constraints for inventory rolling over time periods
for p, s, t in expected_stocks.keys():
    if t == 1: # if p, s has not recorded current stock, we use the current stock as 0
        model.add(expected_stocks[p, s, 1] == current_stocks.get((p, s), 0) + new_orders[p, s, 1] - met_demand[p, s, 1])
    else: # t > 1 
        model.add(expected_stocks[p,s,t] == expected_stocks[p,s,t-1] + new_orders[p,s,t] - met_demand[p,s,t])

In [138]:
unmet_penalty = 10
max_stock_penalty = 5

objective_terms = []
objective_terms.extend([unmet_penalty * ud_var for ud_var in unmet_demand.values()])
objective_terms.extend([max_stock_penalty * exst_var for exst_var in expected_stocks.values()])

model.minimize(sum(objective_terms))

In [139]:
# --- Solve the model ---
solver.parameters.num_search_workers = 8
solver.parameters.max_time_in_seconds = 120
callback = cp_model.ObjectiveSolutionPrinter()
or_status = solver.SolveWithSolutionCallback(model, callback)
status = solver.StatusName(or_status)

if status in ["OPTIMAL", "FEASIBLE"]:
    print(f'Solution: Total cost = {solver.ObjectiveValue()}')
else:
    print('A solution could not be found, check the problem specification')

Solution 0, time = 0.00 s, objective = 15.0
Solution: Total cost = 15.0


In [140]:
def solution_to_df(solution_dict, key_schema:list[tuple[str, type]], value_schema:tuple[str, type]):
    """
    Convert a solution dictionary to a Polars DataFrame.
    solution_dict: dictionary with keys as tuples and values as integers.
    key_schema: list of tuples defining the schema for the keys.
    value_schema: tuple defining the name and type of the value column.
    
    Example:
    solution_to_df(met_demand_solution, key_schema=[("product", str), ("location", str), ("period_rank", int)], value_schema=("met_demand", int))
    """
    # get list of tuples keys from solution_dict
    df = pl.DataFrame(list(solution_dict.keys()), schema=key_schema, orient="row")
    df = df.with_columns(
        pl.Series(value_schema[0], list(solution_dict.values())).cast(value_schema[1])
    )
    return df

def get_solver_solutions(solver, var_dict):
    """
    Extracts the solution from the solver for the given variable dictionary.
    var_dict: dictionary with variable keys.
    Returns a dictionary with the variable keys and their values.
    """
    return {keys: solver.Value(var_dict[keys]) for keys in var_dict.keys()}

In [141]:

met_demand_solution = get_solver_solutions(solver, met_demand)
unmet_demand_solution = get_solver_solutions(solver, unmet_demand)
expected_stocks_solution = get_solver_solutions(solver, expected_stocks)
new_orders_solution = get_solver_solutions(solver, new_orders)

In [153]:
df_met_demand = solution_to_df(
    met_demand_solution, 
    key_schema=[("p_id", pl.Int32), ("s_id", pl.Int32), ("period_rank", pl.Int32)], 
    value_schema=("met_demand", pl.Int32)
)
df_unmet_demand = solution_to_df(
    unmet_demand_solution,
    key_schema=[("p_id", pl.Int32), ("s_id", pl.Int32), ("period_rank", pl.Int32)], 
    value_schema=("unmet_demand", pl.Int32)
)

df_expected_stocks = solution_to_df(
    expected_stocks_solution, 
    key_schema=[("p_id", pl.Int32), ("s_id", pl.Int32), ("period_rank", pl.Int32)], 
    value_schema=("ending_inventory", pl.Int32)
)
# .join(
#     df_periods, 
#     left_on="period_rank", 
#     right_on="period_rank", 
#     how="left"
# ).drop("period_rank").rename({"period": "exsk_period"})

df_new_orders = solution_to_df(
    new_orders_solution, 
    key_schema=[("p_id", pl.Int32), ("s_id", pl.Int32), ("period_rank", pl.Int32)], 
    value_schema=("recommended_orders", pl.Int32)
)

In [ ]:
with duckdb.connect("./data/duckdb.db") as con:
    # con.execute("CREATE OR REPLACE TABLE met_demand AS SELECT * FROM df_met_demand")
    # con.execute("CREATE OR REPLACE TABLE unmet_demand AS SELECT * FROM df_unmet_demand")
    con.execute(
        """
        CREATE SEQUENCE IF NOT EXISTS expected_stocks_id_seq START 1;

        CREATE TABLE IF NOT EXISTS expected_stocks (
            exsk_id INTEGER PRIMARY KEY DEFAULT nextval('expected_stocks_id_seq'),
            exsk_p_id INTEGER,
            exsk_s_id INTEGER,
            exsk_period DATE,
            -- exsk_starting_inventory INTEGER CHECK (exsk_starting_inventory >= 0),
            exsk_ending_inventory INTEGER CHECK (exsk_ending_inventory >= 0),
            UNIQUE (exsk_p_id, exsk_s_id, exsk_period),
            FOREIGN KEY (exsk_p_id) REFERENCES products(p_id),
            FOREIGN KEY (exsk_s_id) REFERENCES stores(s_id) 
        );
        """
    )
    con.execute("INSERT INTO expected_stocks (exsk_p_id, exsk_s_id, exsk_period, exsk_ending_inventory) SELECT exsk_p_id, exsk_s_id, exsk_period, exsk_ending_inventory FROM df_expected_inventory")
    # con.execute("CREATE OR REPLACE TABLE new_orders AS SELECT * FROM df_new_orders")

In [58]:
with duckdb.connect("./data/duckdb.db") as con:
    df_exp = con.execute("SELECT * FROM expected_stocks").pl()

In [160]:
# df_expected_stocks = 
df_output = pl.concat([df_current_stocks, df_expected_stocks], how="vertical") \
        .with_columns(
            starting_inventory=pl.col("ending_inventory").shift(1) \
                .over("p_id", "s_id", order_by="period_rank")
        ).filter(pl.col("period_rank") > 0) \
        .join(df_met_demand, on=["p_id", "s_id", "period_rank"], how="left") \
        .join(df_unmet_demand, on=["p_id", "s_id", "period_rank"], how="left") \
        .join(df_new_orders, on=["p_id", "s_id", "period_rank"], how="left") \
        .join(df_periods, on="period_rank", how="left") \
        .drop("period_rank")
# df_expected_inventory

In [161]:
# I will name this table as procurement_plans 
df_output.select(pl.all().name.prefix("pcpl_"))
# - "",


pcpl_p_id,pcpl_s_id,pcpl_ending_inventory,pcpl_starting_inventory,pcpl_met_demand,pcpl_unmet_demand,pcpl_recommended_orders,pcpl_period
i32,i32,i32,i32,i32,i32,i32,date
1,2,0,5,10,0,5,2024-01-01
1,2,0,0,10,0,10,2024-01-02
1,2,0,0,10,0,10,2024-01-03
1,2,0,0,5,0,5,2024-01-04
1,2,0,0,5,0,5,2024-01-05
…,…,…,…,…,…,…,…
4,1,0,0,12,0,12,2024-01-06
4,2,0,0,5,0,5,2024-01-06
4,3,0,0,10,0,10,2024-01-06


In [172]:
import plotly.graph_objects as go

def plot_inventory_balance(df_output, agg_level=None):
    """
    Visualiza el balance de inventario agregando por el nivel deseado (agg_level).
    agg_level: lista de columnas para agrupar, por ejemplo ["product"], ["location"], ["product", "location"], o None/[] para total.
    """

    # Columnas a sumar
    cols_to_sum = ["starting_inventory", "recommended_orders", "met_demand"]

    # Si agg_level es None o vacío, solo agrupa por periodo
    if not agg_level:
        group_cols = ["period"]
    else:
        group_cols = agg_level + ["period"]

    df_plot = df_output.group_by(group_cols).agg([
        pl.col(c).sum().alias(c) for c in cols_to_sum if c in df_output.columns
    ]).sort("period")

    # Si hay agregación total, solo una gráfica
    if not agg_level:
        title = "Total"
        figs = [("", df_plot)]
    else:
        figs = list(df_plot.group_by(agg_level))

    for keys, subdf in figs:
        if agg_level:
            title = " - ".join(f"{k}: {v}" for k, v in zip(agg_level, keys))
        fig = go.Figure()

        if "starting_inventory" in subdf.columns:
            fig.add_trace(go.Bar(
                x=subdf["period"].to_list(),
                y=subdf["starting_inventory"].to_list(),
                name='Initial Inventory',
                marker_color='rgb(158,202,225)',
                # marker_line_color='rgb(8,48,107)',
            ))

        if "recommended_orders" in subdf.columns:
            fig.add_trace(go.Bar(
                x=subdf["period"].to_list(),
                y=subdf["recommended_orders"].to_list(),
                name='Expected New Orders',
                marker_color='rgb(158,100,225)',
                # marker_line_color='rgb(8,48,107)',
            ))

        if "met_demand" in subdf.columns:
            fig.add_trace(go.Bar(
                x=subdf["period"].to_list(),
                y=[-v for v in subdf["met_demand"].to_list()],
                name='Expected Demand Fulfilled',
                marker_color='green',
                # marker_color='rgb(158,100,225)',
                # marker_line_color='rgb(8,107,48)',
            ))
        #marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)'
        fig.update_traces(marker_line_width=1.5, opacity=0.6)

        fig.update_layout(
            title=f'Inventory Balance ({title})',
            barmode='relative',
            xaxis_title='Period',
            yaxis_title='Quantity',
            legend_title='Legend'
        )
        fig.show()

In [173]:
plot_inventory_balance(df_output, )# agg_level=["location", "product"] )

In [ ]:
import plotly.graph_objects as go

def plot_demand_forecast(df_output, agg_level=None):
    """
    Visualiza df_output agregando por el nivel deseado (agg_level).
    agg_level: lista de columnas para agrupar, por ejemplo ["product"], ["location"], ["product", "location"], o None/[] para total.
    """
    # Asegúrate de que 'period' es string para el eje x
    df_plot = df_output.with_columns(
        pl.col("period").alias("period_str")#.dt.strftime("%Y-%m-%d").alias("period_str")
    )

    # Columnas a sumar
    cols_to_sum = ["mean", "met_demand", "unmet_demand"] #, "quantity", "ending_inventory"]

    # Si agg_level es None o vacío, solo agrupa por periodo
    if not agg_level:
        group_cols = ["period_str"]
    else:
        group_cols = agg_level + ["period_str"]

    df_plot = df_plot.group_by(group_cols).agg([
        pl.col(c).sum().alias(c) for c in cols_to_sum if c in df_plot.columns
    ]).sort("period_str")

    # Si hay agregación total, solo una gráfica
    if not agg_level:
        title = "Total"
        figs = [("", df_plot)]
    else:
        figs = list(df_plot.group_by(agg_level))

    for keys, subdf in figs:
        if agg_level:
            title = " - ".join(f"{k}: {v}" for k, v in zip(agg_level, keys))
        fig = go.Figure()

        if "mean" in subdf.columns:
            fig.add_trace(go.Scatter(
                x=subdf["period_str"].to_list(),
                y=subdf["mean"].to_list(),
                mode='lines+markers',
                name='Demand Forecast',
                line=dict(color='blue')
            ))
        if "met_demand" in subdf.columns:
            fig.add_trace(go.Bar(
                x=subdf["period_str"].to_list(),
                y=subdf["met_demand"].to_list(),
                name='Expected Demand Fulfilled',
                marker_color='green'
            ))
        if "unmet_demand" in subdf.columns:
            fig.add_trace(go.Bar(
                x=subdf["period_str"].to_list(),
                y=subdf["unmet_demand"].to_list(),
                name='Expected Demand Unfulfilled',
                marker_color='red'
            ))
        if "quantity" in subdf.columns:
            fig.add_trace(go.Bar(
                x=subdf["period_str"].to_list(),
                y=subdf["quantity"].to_list(),
                name='New Orders',
                marker_color='orange'
            ))
        if "ending_inventory" in subdf.columns:
            fig.add_trace(go.Bar(
                x=subdf["period_str"].to_list(),
                y=subdf["ending_inventory"].to_list(),
                name='Ending Inventory',
                marker_color='purple'
            ))

        fig.update_layout(
            title=f'Demand & Inventory ({title})',
            barmode='stack',
            xaxis_title='Period',
            yaxis_title='Quantity',
            legend_title='Legend'
        )
        fig.show()

In [ ]:
plot_demand_forecast(df_output, agg_level=None)